In [1]:
%%writefile eda_1sec_lob.py
"""
eda_1sec_lob.py

Exploratory Data Analysis for BTC 1-second limit order book data.

- Loads BTC_1sec.csv
- Checks time deltas and gaps
- Computes 1-second log returns and realized variance
- Constructs a simple OFI (order flow imbalance) signal
- Prints basic stats and autocorrelations
- Saves several diagnostic plots as PNG files
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



DATA_FILE = "/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv"


def load_data(path: str = DATA_FILE) -> pd.DataFrame:
    print(f"Loading {path} ...")
    df = pd.read_csv(path)
    # Drop index column if present
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    # Parse time
    df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
    df = df.sort_values("system_time").reset_index(drop=True)
    return df


def check_time_structure(df: pd.DataFrame):
    print("\n=== Time structure ===")
    dt = df["system_time"].diff()
    # Most common time delta
    mode_delta = dt.value_counts().idxmax()
    print(f"Most common time delta between rows: {mode_delta}")

    # Show unusual gaps (> 2 seconds)
    large_gaps = dt[dt > pd.Timedelta(seconds=2)]
    if len(large_gaps) == 0:
        print("No large gaps (> 2s) detected.")
    else:
        print(f"Found {len(large_gaps)} large gaps (> 2s). Example:")
        print(large_gaps.head())


def compute_returns_and_rv(df: pd.DataFrame) -> pd.DataFrame:
    print("\nComputing 1-second log returns and realized variance...")
    df = df.copy()
    df["mid_log"] = np.log(df["midpoint"])
    df["r_1s"] = df["mid_log"].diff()
    df["rv_1s"] = df["r_1s"] ** 2

    # Longer-horizon realized variances
    for window in [5, 30, 60, 300]:
        df[f"rv_{window}s"] = df["rv_1s"].rolling(window=window, min_periods=window).sum()

    return df


def build_simple_ofi(df: pd.DataFrame, n_levels: int = 5) -> pd.DataFrame:
    """
    Build a simple OFI (Order Flow Imbalance) using changes in top-n bid/ask notionals.

    OFI_t = sum_k Δ(bids_notional_k) - sum_k Δ(asks_notional_k)
    """
    print(f"\nConstructing simple OFI using top {n_levels} levels...")
    df = df.copy()

    bid_cols = [f"bids_notional_{k}" for k in range(n_levels)]
    ask_cols = [f"asks_notional_{k}" for k in range(n_levels)]

    missing = [c for c in bid_cols + ask_cols if c not in df.columns]
    if missing:
        print(f"WARNING: Missing columns for OFI: {missing}")
        print("OFI will be NaN.")
        df["ofi_top"] = np.nan
        return df

    d_bids = df[bid_cols].diff()
    d_asks = df[ask_cols].diff()
    df["ofi_top"] = d_bids.sum(axis=1) - d_asks.sum(axis=1)
    return df


def basic_stats(df: pd.DataFrame):
    print("\n=== Basic stats (selected columns) ===")
    cols = ["midpoint", "spread", "buys", "sells", "r_1s", "rv_1s", "rv_5s", "rv_30s", "ofi_top"]
    cols = [c for c in cols if c in df.columns]
    print(df[cols].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T)


def autocorr(x: pd.Series, lags):
    res = {}
    for lag in lags:
        if lag <= 0 or lag >= len(x):
            res[lag] = np.nan
        else:
            res[lag] = x.autocorr(lag)
    return res


def check_autocorrelations(df: pd.DataFrame):
    print("\n=== Autocorrelations ===")
    # Drop NaNs at the start
    rv = df["rv_1s"].dropna()
    ofi = df["ofi_top"].dropna()

    for name, series in [("rv_1s", rv), ("ofi_top", ofi)]:
        print(f"\n{name} autocorrelations:")
        ac = autocorr(series, lags=[1, 5, 10, 30, 60])
        for lag, val in ac.items():
            print(f"  lag {lag:2d}: {val:.4f}")


def plot_mid_and_spread(df: pd.DataFrame, out_prefix: str = "btc_1sec"):
    print("\nPlotting midprice & spread (downsampled)...")
    # Downsample to 1 minute for readability
    tmp = df[["system_time", "midpoint", "spread"]].set_index("system_time")
    res = tmp.resample("1min").agg({"midpoint": "last", "spread": "median"})

    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(res.index, res["midpoint"], label="midpoint")
    ax1.set_ylabel("Midpoint price")
    ax1.set_title("Midpoint (1-min sampled)")
    ax1.grid(True)
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_midpoint_1min.png")
    plt.close(fig)

    fig, ax2 = plt.subplots(figsize=(12, 4))
    ax2.plot(res.index, res["spread"], label="spread (median)", alpha=0.8)
    ax2.set_ylabel("Spread")
    ax2.set_title("Spread (1-min median)")
    ax2.grid(True)
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_spread_1min.png")
    plt.close(fig)


def plot_depth_profile(df: pd.DataFrame, out_prefix: str = "btc_1sec"):
    print("\nPlotting average depth distance profile...")
    # Use a random subset to reduce memory
    sample = df.sample(n=min(50000, len(df)), random_state=42)

    bid_dist_cols = [c for c in df.columns if c.startswith("bids_distance_")]
    ask_dist_cols = [c for c in df.columns if c.startswith("asks_distance_")]

    if not bid_dist_cols or not ask_dist_cols:
        print("No distance columns found, skipping depth profile plot.")
        return

    # Sort by level index (0..14)
    bid_dist_cols = sorted(bid_dist_cols, key=lambda x: int(x.split("_")[-1]))
    ask_dist_cols = sorted(ask_dist_cols, key=lambda x: int(x.split("_")[-1]))

    bid_mean = sample[bid_dist_cols].mean()
    ask_mean = sample[ask_dist_cols].mean()

    levels = np.arange(len(bid_dist_cols))

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(levels, bid_mean.values, marker="o", label="bids_distance (mean)")
    ax.plot(levels, ask_mean.values, marker="o", label="asks_distance (mean)")
    ax.set_xlabel("Level")
    ax.set_ylabel("Relative distance to mid")
    ax.set_title("Average LOB distance profile (bids vs asks)")
    ax.legend()
    ax.grid(True)
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_depth_distance_profile.png")
    plt.close(fig)


def plot_histograms(df: pd.DataFrame, out_prefix: str = "btc_1sec"):
    print("\nPlotting histograms for RV and volumes...")
    # rv_1s
    rv = df["rv_1s"].dropna()
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.hist(rv, bins=100, alpha=0.8)
    ax.set_yscale("log")
    ax.set_title("Histogram of 1s realized variance (log y-scale)")
    ax.set_xlabel("rv_1s")
    ax.set_ylabel("Count (log)")
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_hist_rv_1s.png")
    plt.close(fig)

    # buys and sells
    for col in ["buys", "sells"]:
        if col in df.columns:
            x = df[col].dropna()
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.hist(x, bins=100, alpha=0.8)
            ax.set_yscale("log")
            ax.set_title(f"Histogram of {col} (log y-scale)")
            ax.set_xlabel(col)
            ax.set_ylabel("Count (log)")
            plt.tight_layout()
            plt.savefig(f"{out_prefix}_hist_{col}.png")
            plt.close(fig)


def plot_ofi_vs_future_rv(df: pd.DataFrame, out_prefix: str = "btc_1sec"):
    print("\nPlotting OFI vs next 1s RV (scatter)...")
    if "ofi_top" not in df.columns:
        print("OFI not found, skipping OFI scatter plot.")
        return

    df = df.copy()
    # Future 1s RV
    df["rv_1s_future"] = df["rv_1s"].shift(-1)

    mask = df["ofi_top"].notna() & df["rv_1s_future"].notna()
    sample = df.loc[mask, ["ofi_top", "rv_1s_future"]]

    # Subsample for plotting
    if len(sample) > 50000:
        sample = sample.sample(n=50000, random_state=42)

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(sample["ofi_top"], sample["rv_1s_future"], s=2, alpha=0.3)
    ax.set_title("OFI (top 5 levels) vs next 1s realized variance")
    ax.set_xlabel("OFI (Δbids_notional - Δasks_notional)")
    ax.set_ylabel("rv_1s (future)")
    ax.grid(True)
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_scatter_ofi_vs_rv_future.png")
    plt.close(fig)


def main():
    df = load_data()
    print(f"Loaded dataset: {df.shape[0]} rows, {df.shape[1]} columns")

    check_time_structure(df)

    # Compute returns & RV
    df = compute_returns_and_rv(df)

    # Build simple OFI
    df = build_simple_ofi(df, n_levels=5)

    # Basic stats
    basic_stats(df)

    # Autocorrelations
    check_autocorrelations(df)

    # Plots
    plot_mid_and_spread(df)
    plot_depth_profile(df)
    plot_histograms(df)
    plot_ofi_vs_future_rv(df)

    print("\nEDA complete. Check the generated PNG files.")


if __name__ == "__main__":
    main()

Writing eda_1sec_lob.py


In [2]:
!python eda_1sec_lob.py

Loading /kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv ...
Loaded dataset: 1030728 rows, 155 columns

=== Time structure ===
Most common time delta between rows: 0 days 00:00:01
Found 37 large gaps (> 2s). Example:
16038    0 days 00:00:02.504599
88034    0 days 00:00:30.000617
102405   0 days 00:00:02.269072
145602   0 days 00:00:02.303479
174400   0 days 00:00:02.110439
Name: system_time, dtype: timedelta64[ns]

Computing 1-second log returns and realized variance...

Constructing simple OFI using top 5 levels...

=== Basic stats (selected columns) ===
              count          mean  ...           99%           max
midpoint  1030728.0  5.997507e+04  ...  6.443001e+04  6.489675e+04
spread    1030728.0  1.314033e+00  ...  1.550000e+01  1.245100e+03
buys      1030728.0  6.060058e+03  ...  1.143633e+05  4.060005e+06
sells     1030728.0  5.278900e+03  ...  1.090945e+05  5.215817e+06
r_1s      1030727.0  1.420149e-08  ...  3.210862e-04  1.112356e-02
rv_1s     103

In [18]:
%%writefile feature_builder_1sec.py
#!/usr/bin/env python3
"""
feature_builder_1sec.py


Input:
    /kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv
    or BTC_1sec.csv in the working directory.

Output:
    BTC_1sec_features.csv

Contents:
    - Cleaned time index
    - 1s log returns and RV
    - Forward RV targets: y_rv_1s, y_rv_5s, y_rv_10s (raw + log)
    - OFI features (top 1/5/10 levels)
    - Depth imbalance (top 1/5/10)
    - Microprice & microprice pressure
    - Rolling RV, volume, spread features
    - Simple queue dynamics (limit vs cancel flows)
        * LOB rebalancing (change in imbalance/depth)
        * LOB pressure ratios (bid/ask depth share)
        * Volatility-adjusted queue/imbalance
        * Liquidity fade metrics
        * Market order impact proxies
"""

import numpy as np
import pandas as pd


DATA_FILE = "/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv"

def load_raw(path: str = DATA_FILE) -> pd.DataFrame:
    print(f"Loading raw data from {path} ...")
    df = pd.read_csv(path)

    # Drop index-like column if present
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    # Parse time and sort
    df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
    df = df.sort_values("system_time").reset_index(drop=True)

    print(f"Loaded dataset: {df.shape[0]} rows, {df.shape[1]} columns")
    return df


def add_basic_price_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    print("\nAdding 1s log returns and RV...")
    df["mid_log"] = np.log(df["midpoint"])
    df["r_1s"] = df["mid_log"].diff()
    df["rv_1s"] = df["r_1s"] ** 2

    return df


def add_forward_targets(df: pd.DataFrame, horizons=(1, 5, 10)) -> pd.DataFrame:
    """
    Add forward realized variance targets for given horizons (in seconds).

    For H in horizons:
      y_rv_{H}s = sum_{t..t+H-1} rv_1s (shifted so value at time t
                   is the RV over [t, t+H] as rows)
    """
    df = df.copy()
    print("\nAdding forward RV targets...")

    eps = 1e-18

    for H in horizons:
        roll = df["rv_1s"].rolling(window=H, min_periods=H).sum()
        # sum rv_1s from t..t+H-1; place at time t
        y_rv = roll.shift(-H + 1)

        name = f"y_rv_{H}s"
        log_name = f"{name}_log"

        df[name] = y_rv
        df[log_name] = np.log(y_rv + eps)

        print(f"  Added {name} and {log_name}")

    return df


def add_ofi_features(df: pd.DataFrame, levels=(1, 5, 10)) -> pd.DataFrame:
    """
    OFI (Order Flow Imbalance) using changes in bids_notional_k and asks_notional_k.

    For each N in levels:
      ofi_topN_t = sum_{k < N} Δ(bids_notional_k) - sum_{k < N} Δ(asks_notional_k)
    """
    df = df.copy()
    print("\nAdding OFI features...")

    max_level = 0
    # detect how many levels we actually have
    for k in range(100):
        if f"bids_notional_{k}" in df.columns:
            max_level = k
        else:
            break
    L = max_level + 1
    print(f"Detected {L} bid/ask notional levels (0..{max_level}).")

    for N in levels:
        N_eff = min(N, L)
        bid_cols = [f"bids_notional_{k}" for k in range(N_eff)]
        ask_cols = [f"asks_notional_{k}" for k in range(N_eff)]

        missing = [c for c in bid_cols + ask_cols if c not in df.columns]
        if missing:
            print(f"  WARNING: missing columns for OFI N={N}: {missing}")
            continue

        d_bids = df[bid_cols].diff()
        d_asks = df[ask_cols].diff()
        df[f"ofi_top{N_eff}"] = d_bids.sum(axis=1) - d_asks.sum(axis=1)
        print(f"  Added ofi_top{N_eff}")

    return df


def add_imbalance_features(df: pd.DataFrame, levels=(1, 5, 10)) -> pd.DataFrame:
    """
    Depth imbalance over top N levels:

      imb_topN = (sum_k bid_notional_k - sum_k ask_notional_k) /
                 (sum_k bid_notional_k + sum_k ask_notional_k)
    """
    df = df.copy()
    print("\nAdding depth imbalance features...")

    max_level = 0
    for k in range(100):
        if f"bids_notional_{k}" in df.columns:
            max_level = k
        else:
            break
    L = max_level + 1

    eps = 1e-18

    for N in levels:
        N_eff = min(N, L)
        bid_cols = [f"bids_notional_{k}" for k in range(N_eff)]
        ask_cols = [f"asks_notional_{k}" for k in range(N_eff)]

        missing = [c for c in bid_cols + ask_cols if c not in df.columns]
        if missing:
            print(f"  WARNING: missing columns for imbalance N={N}: {missing}")
            continue

        bid_sum = df[bid_cols].sum(axis=1)
        ask_sum = df[ask_cols].sum(axis=1)
        num = bid_sum - ask_sum
        den = bid_sum + ask_sum + eps
        df[f"imb_top{N_eff}"] = num / den
        print(f"  Added imb_top{N_eff}")

    return df


def add_microprice_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Approximate best bid/ask from midpoint and distance_0 to build microprice:

      bid_0 = mid * (1 + bids_distance_0)
      ask_0 = mid * (1 + asks_distance_0)

    microprice = (ask_0 * bid_notional_0 + bid_0 * ask_notional_0) /
                 (bid_notional_0 + ask_notional_0)

    microprice_disp = (microprice - midpoint) / midpoint
    """
    df = df.copy()
    print("\nAdding microprice features...")

    required = ["midpoint", "bids_distance_0", "asks_distance_0",
                "bids_notional_0", "asks_notional_0"]
    if any(c not in df.columns for c in required):
        print("  Missing columns for microprice; skipping.")
        return df

    mid = df["midpoint"]
    bid0 = mid * (1.0 + df["bids_distance_0"])
    ask0 = mid * (1.0 + df["asks_distance_0"])

    bid_vol = df["bids_notional_0"]
    ask_vol = df["asks_notional_0"]
    denom = bid_vol + ask_vol

    micro = (ask0 * bid_vol + bid0 * ask_vol) / denom.replace(0, np.nan)
    df["microprice"] = micro
    df["microprice_disp"] = (micro - mid) / mid

    print("  Added microprice and microprice_disp")
    return df


def add_rolling_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add simple rolling features over 5s, 30s, 60s windows:
      - sum/mean of rv_1s
      - sum/mean of buys, sells
      - mean of spread
      - std of r_1s
    """
    df = df.copy()
    print("\nAdding rolling features (5s, 30s, 60s)...")

    windows = [5, 30, 60]

    for w in windows:
        w_str = f"{w}s"

        # Volatility / RV
        df[f"rv_{w_str}_past"] = df["rv_1s"].rolling(window=w, min_periods=w).sum()
        df[f"rv_{w_str}_mean_past"] = df["rv_1s"].rolling(window=w, min_periods=w).mean()

        # Volume
        df[f"buys_sum_{w_str}"] = df["buys"].rolling(window=w, min_periods=w).sum()
        df[f"sells_sum_{w_str}"] = df["sells"].rolling(window=w, min_periods=w).sum()
        df[f"net_flow_{w_str}"] = df[f"buys_sum_{w_str}"] - df[f"sells_sum_{w_str}"]

        # Spread stats
        df[f"spread_mean_{w_str}"] = df["spread"].rolling(window=w, min_periods=w).mean()

        # Return std
        df[f"r_1s_std_{w_str}"] = df["r_1s"].rolling(window=w, min_periods=w).std()

        print(f"  Added rolling features for window={w}s")

    return df


def add_queue_features(df: pd.DataFrame, levels=(1, 5)) -> pd.DataFrame:
    """
    Simple queue dynamics from limit and cancel notional:

      sum_limit_topN_bids, sum_cancel_topN_bids,
      sum_limit_topN_asks, sum_cancel_topN_asks,
      net_liq_change_topN = (limit_bids + limit_asks) - (cancel_bids + cancel_asks)
    """
    df = df.copy()
    print("\nAdding queue dynamics features...")

    # detect max level for these notional types
    def max_level_for(prefix):
        for k in range(100):
            if f"{prefix}_{k}" not in df.columns:
                return k - 1
        return 99

    max_bid_lim = max_level_for("bids_limit_notional")
    max_ask_lim = max_level_for("asks_limit_notional")
    max_bid_can = max_level_for("bids_cancel_notional")
    max_ask_can = max_level_for("asks_cancel_notional")

    for N in levels:
        # bids/asks limit/cancel
        bid_lim_cols = [f"bids_limit_notional_{k}" for k in range(min(N, max_bid_lim + 1))]
        ask_lim_cols = [f"asks_limit_notional_{k}" for k in range(min(N, max_ask_lim + 1))]
        bid_can_cols = [f"bids_cancel_notional_{k}" for k in range(min(N, max_bid_can + 1))]
        ask_can_cols = [f"asks_cancel_notional_{k}" for k in range(min(N, max_ask_can + 1))]

        # Check if any exist
        if not bid_lim_cols or not any(c in df.columns for c in bid_lim_cols + ask_lim_cols):
            print(f"  No limit_notional columns found for top{N}; skipping queue features.")
            continue

        def safe_sum(cols):
            cols = [c for c in cols if c in df.columns]
            if not cols:
                return pd.Series(0.0, index=df.index)
            return df[cols].sum(axis=1)

        lim_bids = safe_sum(bid_lim_cols)
        lim_asks = safe_sum(ask_lim_cols)
        can_bids = safe_sum(bid_can_cols)
        can_asks = safe_sum(ask_can_cols)

        df[f"limit_sum_top{N}_bids"] = lim_bids
        df[f"limit_sum_top{N}_asks"] = lim_asks
        df[f"cancel_sum_top{N}_bids"] = can_bids
        df[f"cancel_sum_top{N}_asks"] = can_asks

        df[f"net_liq_change_top{N}"] = (lim_bids + lim_asks) - (can_bids + can_asks)

        print(f"  Added queue features for top{N}")

    return df


def add_style_features(df: pd.DataFrame, levels=(1, 5, 10)) -> pd.DataFrame:
    """
      - LOB rebalancing: change in imbalance / depth
      - LOB pressure ratios: bid vs ask share of depth
      - Volatility-adjusted imbalance / queue signals
      - Liquidity fade: how quickly depth disappears
      - Market order impact proxies
    """
    df = df.copy()
    print("\nAdding microstructure features...")
    eps = 1e-18

    # Helper to sum depth safely
    def safe_sum(cols):
        cols = [c for c in cols if c in df.columns]
        if not cols:
            return pd.Series(0.0, index=df.index)
        return df[cols].sum(axis=1)

    # Depth-based features for different top-N levels
    max_level = 0
    for k in range(100):
        if f"bids_notional_{k}" in df.columns:
            max_level = k
        else:
            break
    L = max_level + 1

    for N in levels:
        N_eff = min(N, L)
        bid_cols = [f"bids_notional_{k}" for k in range(N_eff)]
        ask_cols = [f"asks_notional_{k}" for k in range(N_eff)]

        depth_bid = safe_sum(bid_cols)
        depth_ask = safe_sum(ask_cols)
        depth_tot = depth_bid + depth_ask

        df[f"depth_bid_top{N_eff}"] = depth_bid
        df[f"depth_ask_top{N_eff}"] = depth_ask
        df[f"depth_tot_top{N_eff}"] = depth_tot

        # LOB pressure ratios (bid / total, ask / total)
        df[f"press_bid_top{N_eff}"] = depth_bid / (depth_tot + eps)
        df[f"press_ask_top{N_eff}"] = depth_ask / (depth_tot + eps)

        # LOB rebalancing: change in imbalance over time
        if f"imb_top{N_eff}" in df.columns:
            df[f"rebal_imb_top{N_eff}"] = df[f"imb_top{N_eff}"].diff()
        else:
            net_depth = depth_bid - depth_ask
            df[f"rebal_imb_top{N_eff}"] = (net_depth / (depth_tot + eps)).diff()

        # Liquidity fade: loss of total depth normalized by previous depth
        depth_diff = depth_tot.diff()
        df[f"liq_fade_top{N_eff}"] = -np.minimum(depth_diff, 0.0) / (depth_tot.shift(1) + eps)

        # Volatility-adjusted imbalance (strong around bursts)
        if f"imb_top{N_eff}" in df.columns:
            df[f"imb_voladj_top{N_eff}"] = df[f"imb_top{N_eff}"] / np.sqrt(df["rv_1s"] + eps)

    # Market order impact proxies (from buys/sells)
    if {"buys", "sells"}.issubset(df.columns):
        # Intensity scaled by price (approx "volume per dollar")
        df["mo_intensity"] = (df["buys"] + df["sells"]) / (df["midpoint"] + eps)

        # Signed imbalance of market orders
        df["mo_imbalance"] = (df["buys"] - df["sells"]) / (df["buys"] + df["sells"] + eps)

        # Volatility-adjusted market order imbalance
        df["mo_imb_voladj"] = df["mo_imbalance"] / np.sqrt(df["rv_1s"] + eps)

        # Simple impact proxy: MO imbalance interacting with recent return
        df["mo_price_impact_proxy"] = df["mo_imbalance"] * df["r_1s"].rolling(3, min_periods=1).sum()

        print("  Added market order impact proxies (mo_intensity, mo_imbalance, mo_imb_voladj, mo_price_impact_proxy)")

    return df


def finalize_and_save(df: pd.DataFrame, out_file: str = "BTC_1sec_features.csv"):
    """
    Clean up and save feature dataset.
    We drop rows where forward targets are NaN (at the tail).
    """
    print("\nFinalizing feature dataset...")

    # Require 1s, 5s, and 10s targets
    target_cols = [
        "y_rv_1s", "y_rv_1s_log",
        "y_rv_5s", "y_rv_5s_log",
        "y_rv_10s", "y_rv_10s_log",
    ]
    existing_targets = [c for c in target_cols if c in df.columns]

    before = len(df)
    if existing_targets:
        df = df.dropna(subset=existing_targets)
        after = len(df)
        print(f"Dropped {before - after} rows with NaN targets; remaining: {after}")
    else:
        print("WARNING: no target columns found; not dropping NaNs on targets.")

    print(f"Saving features to {out_file} ...")
    df.to_csv(out_file, index=False)
    print("Done.")

def add_regime_features(df: pd.DataFrame,
                        n_vol_bins: int = 4,
                        n_liq_bins: int = 3,
                        n_ofi_bins: int = 3) -> pd.DataFrame:
    """
    Add regime-robust features:

      - Realized volatility regime (based on rv_5s_past)
      - Liquidity regime (spread & depth_top1 quantiles)
      - Order-flow regime (OFI intensity on ofi_top1)

    Each regime is added both as an integer label and as one-hot dummies.
    """
    df = df.copy()
    eps = 1e-18
    print("\nAdding regime-robust features...")

    # Safety checks
    required_cols = [
        "rv_5s_past",
        "spread",
        "depth_bid_top1",
        "depth_ask_top1",
        "ofi_top1",
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"  WARNING: missing columns for regime features: {missing}")
        print("  Skipping regime feature construction.")
        return df

    # ---------- 1) Realized volatility regime ----------
    vol_source = df["rv_5s_past"].clip(lower=eps)

    try:
        vol_regime = pd.qcut(
            vol_source,
            q=n_vol_bins,
            labels=False,
            duplicates="drop"
        )
        df["regime_vol"] = vol_regime.astype("float32")
        print(f"  Added regime_vol with up to {n_vol_bins} quantile bins.")
    except ValueError as e:
        print(f"  WARNING: could not compute vol regimes: {e}")
        df["regime_vol"] = np.nan

    # One-hot for vol regime
    if df["regime_vol"].notna().any():
        vol_dummies = pd.get_dummies(
            df["regime_vol"],
            prefix="regime_vol",
            dtype=np.int8
        )
        df = pd.concat([df, vol_dummies], axis=1)
        print(f"  Added {vol_dummies.shape[1]} one-hot columns for regime_vol.")

    # ---------- 2) Liquidity regime ----------
    # (a) Spread regime
    try:
        spread_regime = pd.qcut(
            df["spread"],
            q=n_liq_bins,
            labels=False,
            duplicates="drop"
        )
        df["regime_spread"] = spread_regime.astype("float32")
        print(f"  Added regime_spread with up to {n_liq_bins} bins.")
    except ValueError as e:
        print(f"  WARNING: could not compute spread regimes: {e}")
        df["regime_spread"] = np.nan

    if df["regime_spread"].notna().any():
        spread_dummies = pd.get_dummies(
            df["regime_spread"],
            prefix="regime_spread",
            dtype=np.int8
        )
        df = pd.concat([df, spread_dummies], axis=1)
        print(f"  Added {spread_dummies.shape[1]} one-hot columns for regime_spread.")

    # (b) Depth regime (top-of-book depth)
    depth_top1 = df["depth_bid_top1"] + df["depth_ask_top1"]
    try:
        depth_regime = pd.qcut(
            depth_top1,
            q=n_liq_bins,
            labels=False,
            duplicates="drop"
        )
        df["regime_depth"] = depth_regime.astype("float32")
        print(f"  Added regime_depth with up to {n_liq_bins} bins.")
    except ValueError as e:
        print(f"  WARNING: could not compute depth regimes: {e}")
        df["regime_depth"] = np.nan

    if df["regime_depth"].notna().any():
        depth_dummies = pd.get_dummies(
            df["regime_depth"],
            prefix="regime_depth",
            dtype=np.int8
        )
        df = pd.concat([df, depth_dummies], axis=1)
        print(f"  Added {depth_dummies.shape[1]} one-hot columns for regime_depth.")

    # ---------- 3) Order-flow regime (OFI) ----------
    ofi_source = df["ofi_top1"]

    try:
        ofi_regime = pd.qcut(
            ofi_source,
            q=n_ofi_bins,
            labels=False,
            duplicates="drop"
        )
        df["regime_ofi"] = ofi_regime.astype("float32")
        print(f"  Added regime_ofi with up to {n_ofi_bins} bins.")
    except ValueError as e:
        print(f"  WARNING: could not compute OFI regimes: {e}")
        df["regime_ofi"] = np.nan

    if df["regime_ofi"].notna().any():
        ofi_dummies = pd.get_dummies(
            df["regime_ofi"],
            prefix="regime_ofi",
            dtype=np.int8
        )
        df = pd.concat([df, ofi_dummies], axis=1)
        print(f"  Added {ofi_dummies.shape[1]} one-hot columns for regime_ofi.")

    print("Regime-robust features added.")
    return df

def add_vol_regime(df: pd.DataFrame, q=3):
    """
    Volatility regime based on rolling 60s RV.
    Creates rv_regime in {0,1,2}.
    """
    df = df.copy()
    if "rv_60s_mean_past" not in df.columns:
        print("No rv_60s_mean_past, skipping rv_regime.")
        return df

    rv = df["rv_60s_mean_past"].fillna(method="bfill").fillna(method="ffill")
    df["rv_regime"] = pd.qcut(rv, q=q, labels=False)
    print("Added rv_regime (0=low,1=mid,2=high).")
    return df



def main():
    df = load_raw()

    df = add_basic_price_features(df)
    df = add_forward_targets(df, horizons=(1, 5, 10))
    df = add_ofi_features(df, levels=(1, 5, 10))
    df = add_imbalance_features(df, levels=(1, 5, 10))
    df = add_microprice_features(df)
    df = add_rolling_features(df)
    df = add_queue_features(df, levels=(1, 5))
    df = add_style_features(df, levels=(1, 5, 10))
    df = add_regime_features(df)
    df = add_vol_regime(df)

    finalize_and_save(df)


if __name__ == "__main__":
    main()


Overwriting feature_builder_1sec.py


In [ ]:
!python feature_builder_1sec.py


Loading raw data from /kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1sec.csv ...
Loaded dataset: 1030728 rows, 155 columns

Adding 1s log returns and RV...

Adding forward RV targets...
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
  Added y_rv_1s and y_rv_1s_log
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
  Added y_rv_5s and y_rv_5s_log
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
  Added y_rv_10s and y_rv_10s_log

Adding OFI features...
Detected 15 bid/ask notional levels (0..14).
  Added ofi_top1
  Added ofi_top5
  Added ofi_top10

Adding depth imbalance features...
  Added imb_top1
  Added im

In [5]:
%%writefile train_rv_5s_xgb.py
#!/usr/bin/env python3
"""
train_rv_5s_xgb.py

Train an XGBoost model to predict 5-second realized variance (RV)
using BTC 1-second limit order book features.

Target:
    y_rv_5s_log  (forward 5s RV in log-space, produced by feature_builder_1sec.py)

Baseline:
    log(rv_5s_past + eps)  (past 5s RV in log-space)

Input:
    BTC_1sec_features.csv

Outputs:
    - Printed metrics for baseline and XGB (TRAIN/VALID/TEST)
    - feature_importance_rv_5s_xgb.csv
    - rv_5s_pred_vs_true_test.png
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error


FEATURE_FILE = "BTC_1sec_features.csv"


# ---------- Helpers ----------

def spearman_corr(a, b):
    a_rank = pd.Series(a).rank()
    b_rank = pd.Series(b).rank()
    return a_rank.corr(b_rank)


def eval_log_and_rv(y_log_true, y_log_pred, name="set"):
    eps = 1e-18

    mse_log = mean_squared_error(y_log_true, y_log_pred)
    mae_log = mean_absolute_error(y_log_true, y_log_pred)
    corr_log = np.corrcoef(y_log_true, y_log_pred)[0, 1]

    # Convert back to RV space
    y_rv_true = np.exp(y_log_true) - eps
    y_rv_pred = np.exp(y_log_pred) - eps

    mse_rv = mean_squared_error(y_rv_true, y_rv_pred)
    mae_rv = mean_absolute_error(y_rv_true, y_rv_pred)
    corr_rv_p = np.corrcoef(y_rv_true, y_rv_pred)[0, 1]
    corr_rv_s = spearman_corr(y_rv_true, y_rv_pred)

    print(f"\n=== Performance on {name} ===")
    print("Log-space (y_rv_5s_log):")
    print(f"  RMSE_log: {np.sqrt(mse_log):.6f}")
    print(f"  MAE_log:  {mae_log:.6f}")
    print(f"  Corr_log (Pearson): {corr_log:.4f}")
    print("RV-space (y_rv_5s):")
    print(f"  RMSE_rv:   {np.sqrt(mse_rv):.6e}")
    print(f"  MAE_rv:    {mae_rv:.6e}")
    print(f"  Corr_rv_P: {corr_rv_p:.4f}")
    print(f"  Corr_rv_S: {corr_rv_s:.4f}")

    return {
        "RMSE_log": np.sqrt(mse_log),
        "MAE_log": mae_log,
        "RMSE_rv": np.sqrt(mse_rv),
        "MAE_rv": mae_rv,
        "Corr_log": corr_log,
        "Corr_P": corr_rv_p,
        "Corr_S": corr_rv_s,
    }


def build_xgb():
    
    return XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        min_child_weight=10,
        objective="reg:squarederror",
        tree_method="hist",
        device = "cuda",
        eval_metric="rmse",
        n_jobs=-1,
    )


# ---------- Main ----------

def main():
    print(f"Loading feature file: {FEATURE_FILE}")
    df = pd.read_csv(FEATURE_FILE)

    # Parse time and sort to be safe
    if "system_time" in df.columns:
        df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
        df = df.sort_values("system_time").reset_index(drop=True)

    print(f"Loaded features: {df.shape[0]} rows, {df.shape[1]} columns")

    # We use forward 5s RV produced by the feature builder: y_rv_5s_log
    # And baseline based on rv_5s_past
    eps = 1e-18

    if "y_rv_5s_log" not in df.columns:
        raise ValueError("Column 'y_rv_5s_log' not found in features file.")
    if "rv_5s_past" not in df.columns:
        raise ValueError("Column 'rv_5s_past' not found (needed for baseline).")

    # Drop rows where either target OR past-5s RV is NaN
    before = len(df)
    df = df.dropna(subset=["y_rv_5s_log", "rv_5s_past"]).reset_index(drop=True)
    after = len(df)
    print(f"Dropped {before - after} rows with NaN y_rv_5s_log / rv_5s_past; remaining: {after}")

    # Time-based split: 70% train, 15% valid, 15% test
    n = len(df)
    train_end = int(n * 0.7)
    valid_end = int(n * 0.85)

    train = df.iloc[:train_end]
    valid = df.iloc[train_end:valid_end]
    test = df.iloc[valid_end:]

    print("\nSplit sizes:")
    print(f"  Train: {len(train)}")
    print(f"  Valid: {len(valid)}")
    print(f"  Test:  {len(test)}")

    # Features: exclude time and ALL y_rv_* targets (1s/5s/10s etc.) to avoid leakage.
    exclude_cols = {
        "system_time",
        "y_rv_5s_log",   # target (log)
        "y_rv_5s",       # raw target if present
    }
    # Remove any other y_rv_* columns that might exist (e.g. y_rv_1s, y_rv_10s, etc.)
    exclude_cols |= {c for c in df.columns if c.startswith("y_rv_")}

    # We keep rv_5s_past as a legitimate predictor (it's past-only)
    feature_cols = [c for c in df.columns if c not in exclude_cols]

    print(f"\nUsing {len(feature_cols)} features.")
    # Uncomment to inspect:
    # for c in feature_cols:
    #     print("  ", c)

    X_train = train[feature_cols].values
    y_train = train["y_rv_5s_log"].values

    X_valid = valid[feature_cols].values
    y_valid = valid["y_rv_5s_log"].values

    X_test = test[feature_cols].values
    y_test = test["y_rv_5s_log"].values

    # --- Baseline: use log(rv_5s_past) to predict y_rv_5s_log ---
    rv_5s_past_all = df["rv_5s_past"].values
    rv_5s_past_log_all = np.log(rv_5s_past_all + eps)

    baseline_train = rv_5s_past_log_all[:train_end]
    baseline_valid = rv_5s_past_log_all[train_end:valid_end]
    baseline_test = rv_5s_past_log_all[valid_end:]

    print("\n=== Baseline (log(rv_5s_past) -> y_rv_5s_log) ===")
    _ = eval_log_and_rv(y_train, baseline_train, name="TRAIN (baseline)")
    _ = eval_log_and_rv(y_valid, baseline_valid, name="VALID (baseline)")
    _ = eval_log_and_rv(y_test, baseline_test, name="TEST (baseline)")

    # --- Train XGBoost model ---
    print("\n=== Training XGBoost model for 5s RV ===")
    model = build_xgb()
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=50,  # print eval RMSE every 50 trees
    )

    print("\n=== Evaluating XGBoost ===")
    # Train
    y_train_pred = model.predict(X_train)
    res_train = eval_log_and_rv(y_train, y_train_pred, name="TRAIN (XGB)")

    # Valid
    y_valid_pred = model.predict(X_valid)
    res_valid = eval_log_and_rv(y_valid, y_valid_pred, name="VALID (XGB)")

    # Test
    y_test_pred = model.predict(X_test)
    res_test = eval_log_and_rv(y_test, y_test_pred, name="TEST (XGB)")

    # --- Feature importance ---
    print("\n=== Feature Importance (gain) ===")
    importance = model.get_booster().get_score(importance_type="gain")
    rows = []
    for i, col in enumerate(feature_cols):
        key = f"f{i}"
        gain = importance.get(key, 0.0)
        rows.append((col, gain))
    fi_df = pd.DataFrame(rows, columns=["feature", "gain"]).sort_values(
        "gain", ascending=False
    )
    print(fi_df.head(30))
    fi_df.to_csv("feature_importance_rv_5s_xgb.csv", index=False)
    print("Saved feature_importance_rv_5s_xgb.csv")

    # --- Plot predictions vs true (log-space) on TEST ---
    print("Saving rv_5s_pred_vs_true_test.png ...")
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_test_pred, s=2, alpha=0.3)
    lims = [min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())]
    plt.plot(lims, lims, "r--", linewidth=1)
    plt.xlabel("True y_rv_5s_log")
    plt.ylabel("Predicted y_rv_5s_log")
    plt.title("XGB: True vs Predicted 5s log RV (TEST)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("rv_5s_pred_vs_true_test.png")
    plt.close()
    print("Done.")


if __name__ == "__main__":
    main()


Writing train_rv_5s_xgb.py


In [6]:
!python train_rv_5s_xgb.py


Loading feature file: BTC_1sec_features.csv
Loaded features: 1030718 rows, 246 columns
Dropped 4 rows with NaN y_rv_5s_log / rv_5s_past; remaining: 1030714

Split sizes:
  Train: 721499
  Valid: 154607
  Test:  154608

Using 239 features.

=== Baseline (log(rv_5s_past) -> y_rv_5s_log) ===

=== Performance on TRAIN (baseline) ===
Log-space (y_rv_5s_log):
  RMSE_log: 11.526649
  MAE_log:  6.647893
  Corr_log (Pearson): 0.4488
RV-space (y_rv_5s):
  RMSE_rv:   1.093229e-07
  MAE_rv:    3.664576e-08
  Corr_rv_P: 0.4310
  Corr_rv_S: 0.4495

=== Performance on VALID (baseline) ===
Log-space (y_rv_5s_log):
  RMSE_log: 10.966455
  MAE_log:  6.293342
  Corr_log (Pearson): 0.3998
RV-space (y_rv_5s):
  RMSE_rv:   1.559154e-07
  MAE_rv:    4.218196e-08
  Corr_rv_P: 0.2988
  Corr_rv_S: 0.4119

=== Performance on TEST (baseline) ===
Log-space (y_rv_5s_log):
  RMSE_log: 10.463453
  MAE_log:  5.705383
  Corr_log (Pearson): 0.4246
RV-space (y_rv_5s):
  RMSE_rv:   1.498229e-06
  MAE_rv:    1.553576e-07
 

In [13]:
%%writefile cv_rv_5sec_xgb.py
#!/usr/bin/env python3
"""
cv_rv_5sec_xgb.py

Rolling time-series CV + random hyperparameter search + blend alpha tuning
for 5-second realized variance (RV) prediction on BTC 1-second LOB data.

Pipeline:

1. Load BTC_1sec_features.csv.
2. Target:
       y_rv_5s_log  (forward 5s RV in log-space, from feature_builder_1sec.py)
3. Baseline:
       log(rv_5s_past + eps)  (past 5s RV in log-space)
4. Hold out the last 15% of data as a FINAL TEST set.
5. On the first 85% ("development" data), run K-fold *expanding* CV:
     fold k:
       train: [0 : val_start_k)
       valid: [val_start_k : val_end_k)
6. For each random XGB hyperparameter config:
     - Train on each fold's train, predict on fold valid.
     - Collect baseline and XGB predictions.
     - On concatenated CV preds, search alpha in [0,1] (grid)
       to maximize Corr_rv_P (Pearson corr in RV space).
7. Choose (params, alpha) with best CV Corr_rv_P.
8. Refit best XGB on whole development data.
9. Evaluate baseline, pure XGB, and blend(alpha) on:
     - development segment
     - final TEST segment
10. Fit a **global** isotonic calibrator on DEV blended log-RV and
    evaluate calibrated blend on DEV + TEST.

Outputs:
    - Printed CV summary and TEST performance
    - best_rv_5s_cv_config.json
    - calibrator_rv_5s_isotonic_global.pkl
"""

import json
import pickle
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.isotonic import IsotonicRegression


FEATURE_FILE = "BTC_1sec_features.csv"
EPS = 1e-18


# ---------- Helpers ----------

def spearman_corr(a, b):
    a_rank = pd.Series(a).rank()
    b_rank = pd.Series(b).rank()
    return a_rank.corr(b_rank)


def eval_log_and_rv(y_log_true, y_log_pred, name="set"):
    """Evaluate metrics in log-space and RV-space."""
    mse_log = mean_squared_error(y_log_true, y_log_pred)
    mae_log = mean_absolute_error(y_log_true, y_log_pred)
    corr_log = np.corrcoef(y_log_true, y_log_pred)[0, 1]

    # Back to RV space
    y_rv_true = np.exp(y_log_true) - EPS
    y_rv_pred = np.exp(y_log_pred) - EPS

    mse_rv = mean_squared_error(y_rv_true, y_rv_pred)
    mae_rv = mean_absolute_error(y_rv_true, y_rv_pred)
    corr_rv_p = np.corrcoef(y_rv_true, y_rv_pred)[0, 1]
    corr_rv_s = spearman_corr(y_rv_true, y_rv_pred)

    print(f"\n=== Performance on {name} ===")
    print("Log-space:")
    print(f"  RMSE_log: {np.sqrt(mse_log):.6f}")
    print(f"  MAE_log:  {mae_log:.6f}")
    print(f"  Corr_log (Pearson): {corr_log:.4f}")
    print("RV-space:")
    print(f"  RMSE_rv:   {np.sqrt(mse_rv):.6e}")
    print(f"  MAE_rv:    {mae_rv:.6e}")
    print(f"  Corr_rv_P: {corr_rv_p:.4f}")
    print(f"  Corr_rv_S: {corr_rv_s:.4f}")

    return {
        "RMSE_log": float(np.sqrt(mse_log)),
        "MAE_log": float(mae_log),
        "RMSE_rv": float(np.sqrt(mse_rv)),
        "MAE_rv": float(mae_rv),
        "Corr_log": float(corr_log),
        "Corr_P": float(corr_rv_p),
        "Corr_S": float(corr_rv_s),
    }


def make_folds(n_dev, n_folds=3):
    """
    Create expanding time-series folds on [0, n_dev).

    For fold k:
      valid = [v_start : v_end)
      train = [0 : v_start)
    """
    folds = []
    fold_size = n_dev // (n_folds + 1)
    for k in range(1, n_folds + 1):
        v_start = k * fold_size
        v_end = (k + 1) * fold_size if k < n_folds else n_dev
        folds.append((0, v_start, v_start, v_end))
    return folds


def sample_param_grid(n_configs=10, random_state=42):
    """
    Random hyperparameter sampling for XGB.
    Adjust ranges if you want to push harder.
    """
    rng = np.random.RandomState(random_state)
    configs = []
    for _ in range(n_configs):
        cfg = {
            "n_estimators": int(rng.randint(300, 700)),
            "learning_rate": float(10 ** rng.uniform(-2.0, -0.7)),  # ~[0.01, 0.2]
            "max_depth": int(rng.randint(4, 9)),  # 4..8
            "subsample": float(rng.uniform(0.7, 1.0)),
            "colsample_bytree": float(rng.uniform(0.7, 1.0)),
            "min_child_weight": float(rng.choice([1.0, 5.0, 10.0, 20.0])),
        }
        configs.append(cfg)
    return configs


def build_xgb(params):
    """Create an XGBRegressor from a params dict (GPU-enabled)."""
    return XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",   # you can switch to "gpu_hist" if supported
        device="cuda",
        eval_metric="rmse",
        n_jobs=-1,
        **params,
    )


def blend_and_metric(y_log_true, log_base, log_xgb, alphas):
    """
    For given arrays (true, baseline, xgb) in log-space, evaluate blending:

       y_pred_log(alpha) = alpha * log_base + (1 - alpha) * log_xgb

    Returns:
       best_alpha, metrics_by_alpha (dict list)
    """
    y_rv_true = np.exp(y_log_true) - EPS

    best_alpha = None
    best_corr = -np.inf
    metrics = []

    for alpha in alphas:
        y_log_blend = alpha * log_base + (1.0 - alpha) * log_xgb
        y_rv_blend = np.exp(y_log_blend) - EPS

        corr_p = np.corrcoef(y_rv_true, y_rv_blend)[0, 1]
        rmse_rv = np.sqrt(mean_squared_error(y_rv_true, y_rv_blend))

        metrics.append({
            "alpha": float(alpha),
            "Corr_P": float(corr_p),
            "RMSE_rv": float(rmse_rv),
        })

        if corr_p > best_corr:
            best_corr = corr_p
            best_alpha = alpha

    return best_alpha, metrics


# ---------- Main ----------

def main():
    print(f"Loading feature file: {FEATURE_FILE}")
    df = pd.read_csv(FEATURE_FILE)

    # Sort by time to be safe
    if "system_time" in df.columns:
        df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
        df = df.sort_values("system_time").reset_index(drop=True)

    print(f"Loaded features: {df.shape[0]} rows, {df.shape[1]} columns")

    # --- Target and baseline for 5s RV ---
    if "y_rv_5s_log" not in df.columns:
        raise ValueError("Column 'y_rv_5s_log' not found in features file.")
    if "rv_5s_past" not in df.columns:
        raise ValueError("Column 'rv_5s_past' not found (needed for baseline).")

    # Drop rows where either target or past-5s RV is NaN
    before = len(df)
    df = df.dropna(subset=["y_rv_5s_log", "rv_5s_past"]).reset_index(drop=True)
    after = len(df)
    print(f"Dropped {before - after} rows with NaN y_rv_5s_log / rv_5s_past; remaining: {after}")

    n = len(df)

    # Holdout TEST = last 15%
    test_start = int(n * 0.85)
    dev = df.iloc[:test_start].copy()   # used for CV + final training
    test = df.iloc[test_start:].copy()  # untouched until the end

    print("\nGlobal split:")
    print(f"  DEV:  {len(dev)} rows (for CV + final fit)")
    print(f"  TEST: {len(test)} rows (final holdout)")

    # --- Baseline feature: log(rv_5s_past) ---
    dev_log_rv_past = np.log(dev["rv_5s_past"] + EPS)
    test_log_rv_past = np.log(test["rv_5s_past"] + EPS)

    # Targets
    y_dev = dev["y_rv_5s_log"].values
    y_test = test["y_rv_5s_log"].values

    # --- Define feature columns (exclude time & explicit targets) ---
    exclude_cols = {
        "system_time",
        "y_rv_1s", "y_rv_1s_log",
        "y_rv_5s", "y_rv_5s_log",
        "y_rv_10s", "y_rv_10s_log",
        # safety: if 1s CV script was run and added future targets
        "y_rv_1s_future", "y_rv_1s_future_log",
    }

    feature_cols = [c for c in dev.columns if c not in exclude_cols]
    print(f"\nUsing {len(feature_cols)} features for 5s RV.")

    X_dev = dev[feature_cols].values
    X_test = test[feature_cols].values

    # ---------- CV setup ----------
    n_dev = len(dev)
    n_folds = 3
    folds = make_folds(n_dev, n_folds=n_folds)
    print(f"\nUsing {n_folds}-fold expanding time-series CV on DEV:")
    for i, (tr_start, tr_end, v_start, v_end) in enumerate(folds):
        print(f"  Fold {i+1}: train=[{tr_start}:{tr_end}), valid=[{v_start}:{v_end})")

    # ---------- Random hyperparameter search ----------
    param_grid = sample_param_grid(n_configs=10, random_state=42)
    alpha_grid = np.linspace(0.0, 1.0, 21)  # 0, 0.05, ..., 1.0

    best_overall = {
        "params": None,
        "alpha": None,
        "cv_corr": -np.inf,
        "cv_rmse_rv": None,
    }

    print("\n===== Starting CV hyperparameter + alpha search =====")
    for cfg_idx, params in enumerate(param_grid):
        print(f"\n--- Config {cfg_idx+1}/{len(param_grid)} ---")
        print(params)

        # Collect CV predictions
        cv_y_true = []
        cv_log_base = []
        cv_log_xgb = []

        for fold_idx, (tr_start, tr_end, v_start, v_end) in enumerate(folds):
            print(f"  Fold {fold_idx+1}: training on [{tr_start}:{tr_end}), validating on [{v_start}:{v_end})")

            X_tr = X_dev[tr_start:tr_end]
            y_tr = y_dev[tr_start:tr_end]

            X_val = X_dev[v_start:v_end]
            y_val = y_dev[v_start:v_end]

            # Baseline
            log_base_val = dev_log_rv_past.iloc[v_start:v_end].values

            # Train XGB
            model = build_xgb(params)
            model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_val, y_val)],
                verbose=False,
            )
            log_xgb_val = model.predict(X_val)

            cv_y_true.append(y_val)
            cv_log_base.append(log_base_val)
            cv_log_xgb.append(log_xgb_val)

        # Concatenate over folds
        cv_y_true = np.concatenate(cv_y_true)
        cv_log_base = np.concatenate(cv_log_base)
        cv_log_xgb = np.concatenate(cv_log_xgb)

        # Find best alpha for this config on CV
        best_alpha_cfg, alpha_metrics = blend_and_metric(
            cv_y_true, cv_log_base, cv_log_xgb, alpha_grid
        )

        # Get metrics for that alpha
        y_rv_true = np.exp(cv_y_true) - EPS
        y_rv_blend = np.exp(
            best_alpha_cfg * cv_log_base + (1.0 - best_alpha_cfg) * cv_log_xgb
        ) - EPS

        corr_cv = np.corrcoef(y_rv_true, y_rv_blend)[0, 1]
        rmse_cv = np.sqrt(mean_squared_error(y_rv_true, y_rv_blend))

        print(f"  -> Best alpha (by CV Corr_P): {best_alpha_cfg:.2f}")
        print(f"     CV Corr_rv_P: {corr_cv:.4f}, CV RMSE_rv: {rmse_cv:.4e}")

        if corr_cv > best_overall["cv_corr"]:
            best_overall["params"] = params
            best_overall["alpha"] = float(best_alpha_cfg)
            best_overall["cv_corr"] = float(corr_cv)
            best_overall["cv_rmse_rv"] = float(rmse_cv)

    print("\n===== CV search finished =====")
    print("Best config found:")
    print(json.dumps(best_overall, indent=2))

    # ---------- Final training on full DEV ----------
    print("\n=== Training final model on full DEV with best params ===")
    best_params = best_overall["params"]
    best_alpha = best_overall["alpha"]

    final_model = build_xgb(best_params)
    final_model.fit(
        X_dev,
        y_dev,
        eval_set=[(X_dev, y_dev)],
        verbose=False,
    )

    # Baseline & XGB & blend on DEV
    log_base_dev = dev_log_rv_past.values
    log_xgb_dev = final_model.predict(X_dev)
    log_blend_dev = best_alpha * log_base_dev + (1.0 - best_alpha) * log_xgb_dev

    _ = eval_log_and_rv(y_dev, log_base_dev, name="DEV (baseline)")
    _ = eval_log_and_rv(y_dev, log_xgb_dev, name="DEV (XGB)")
    _ = eval_log_and_rv(y_dev, log_blend_dev, name=f"DEV (blend, alpha={best_alpha:.2f})")

    # ---------- Final evaluation on TEST ----------
    print("\n=== Final evaluation on TEST (holdout) ===")

    # Baseline on TEST
    log_base_test = test_log_rv_past.values

    # XGB on TEST
    log_xgb_test = final_model.predict(X_test)

    # Blended
    log_blend_test = best_alpha * log_base_test + (1.0 - best_alpha) * log_xgb_test

    res_base = eval_log_and_rv(y_test, log_base_test, name="TEST (baseline)")
    res_xgb = eval_log_and_rv(y_test, log_xgb_test, name="TEST (XGB)")
    res_blend = eval_log_and_rv(y_test, log_blend_test, name=f"TEST (blend, alpha={best_alpha:.2f})")

    summary = {
        "best_params": best_params,
        "best_alpha": best_alpha,
        "cv_corr_rv_P": best_overall["cv_corr"],
        "cv_rmse_rv": best_overall["cv_rmse_rv"],
        "test_baseline": res_base,
        "test_xgb": res_xgb,
        "test_blend": res_blend,
    }

    with open("best_rv_5s_cv_config.json", "w") as f:
        json.dump(summary, f, indent=2)
    print("\nSaved best_rv_5s_cv_config.json")

    # ---------- Global isotonic calibration on blended predictions ----------
    print("\n=== GLOBAL isotonic calibration on blended predictions ===")

    iso = IsotonicRegression(out_of_bounds="clip")
    print("Fitting GLOBAL IsotonicRegression calibrator on DEV...")
    iso.fit(log_blend_dev, y_dev)

    log_cal_dev = iso.predict(log_blend_dev)
    log_cal_test = iso.predict(log_blend_test)

    _ = eval_log_and_rv(y_dev, log_cal_dev, name="DEV (global calibrated blend)")
    _ = eval_log_and_rv(y_test, log_cal_test, name="TEST (global calibrated blend)")

    with open("calibrator_rv_5s_isotonic_global.pkl", "wb") as f:
        pickle.dump(iso, f)
    
    print("Saved global calibrator to calibrator_rv_5s_isotonic_global.pkl")


if __name__ == "__main__":
    main()


Overwriting cv_rv_5sec_xgb.py


In [14]:
!python cv_rv_5sec_xgb.py


Loading feature file: BTC_1sec_features.csv
Loaded features: 1030718 rows, 246 columns
Dropped 4 rows with NaN y_rv_5s_log / rv_5s_past; remaining: 1030714

Global split:
  DEV:  876106 rows (for CV + final fit)
  TEST: 154608 rows (final holdout)

Using 239 features for 5s RV.

Using 3-fold expanding time-series CV on DEV:
  Fold 1: train=[0:219026), valid=[219026:438052)
  Fold 2: train=[0:438052), valid=[438052:657078)
  Fold 3: train=[0:657078), valid=[657078:876106)

===== Starting CV hyperparameter + alpha search =====

--- Config 1/10 ---
{'n_estimators': 402, 'learning_rate': 0.1085190249433509, 'max_depth': 6, 'subsample': 0.9339073000818308, 'colsample_bytree': 0.879055047383946, 'min_child_weight': 5.0}
  Fold 1: training on [0:219026), validating on [219026:438052)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:09:53] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might 

In [9]:
%%writefile calibrate_rv_5s.py
#!/usr/bin/env python3
"""
calibrate_rv_5s.py

Post-hoc calibration for 5-second RV predictions.

Pipeline:
  1) Load BTC_1sec_features.csv.
  2) Recreate the 5s DEV / TEST split used in cv_rv_5sec_xgb.py:
       - Drop rows with NaN in y_rv_5s_log or rv_5s_past.
       - DEV = first 85%, TEST = last 15%.
  3) Load best 5s CV config from best_rv_5s_cv_config.json:
       - best_params (XGB hyperparams)
       - best_alpha (blend weight for baseline vs XGB)
  4) Train final XGB on DEV only.
  5) For each DEV point:
       y_true = y_rv_5s_log
       log_base = log(rv_5s_past + eps)
       log_xgb  = XGB prediction
       log_raw  = alpha * log_base + (1 - alpha) * log_xgb
     Fit IsotonicRegression: log_raw -> y_calibrated (in log-space).
  6) Apply calibrator to DEV and TEST predictions.
  7) Compare metrics (raw blend vs calibrated blend) on DEV and TEST.
  8) Save:
       - calibrator_rv_5s_isotonic.pkl
       - calibrator_rv_5s_isotonic.png
"""

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

from xgboost import XGBRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


FEATURE_FILE = "BTC_1sec_features.csv"
CV_CONFIG_FILE = "best_rv_5s_cv_config.json"
EPS = 1e-18


# ---------- Helpers ----------

def spearman_corr(a, b):
    a_rank = pd.Series(a).rank()
    b_rank = pd.Series(b).rank()
    return a_rank.corr(b_rank)


def eval_log_and_rv(y_log_true, y_log_pred, name="set"):
    """Evaluate metrics in log-space and RV-space."""
    mse_log = mean_squared_error(y_log_true, y_log_pred)
    mae_log = mean_absolute_error(y_log_true, y_log_pred)
    corr_log = np.corrcoef(y_log_true, y_log_pred)[0, 1]

    # Back to RV space
    y_rv_true = np.exp(y_log_true) - EPS
    y_rv_pred = np.exp(y_log_pred) - EPS

    mse_rv = mean_squared_error(y_rv_true, y_rv_pred)
    mae_rv = mean_absolute_error(y_rv_true, y_rv_pred)
    corr_rv_p = np.corrcoef(y_rv_true, y_rv_pred)[0, 1]
    corr_rv_s = spearman_corr(y_rv_true, y_rv_pred)

    print(f"\n=== Performance on {name} ===")
    print("Log-space (y_rv_5s_log):")
    print(f"  RMSE_log: {np.sqrt(mse_log):.6f}")
    print(f"  MAE_log:  {mae_log:.6f}")
    print(f"  Corr_log (Pearson): {corr_log:.4f}")
    print("RV-space (y_rv_5s):")
    print(f"  RMSE_rv:   {np.sqrt(mse_rv):.6e}")
    print(f"  MAE_rv:    {mae_rv:.6e}")
    print(f"  Corr_rv_P: {corr_rv_p:.4f}")
    print(f"  Corr_rv_S: {corr_rv_s:.4f}")

    return {
        "RMSE_log": float(np.sqrt(mse_log)),
        "MAE_log": float(mae_log),
        "RMSE_rv": float(np.sqrt(mse_rv)),
        "MAE_rv": float(mae_rv),
        "Corr_log": float(corr_log),
        "Corr_P": float(corr_rv_p),
        "Corr_S": float(corr_rv_s),
    }


def build_xgb(params):
    """Create an XGBRegressor from a params dict."""
    return XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        device="cuda",          # GPU
        eval_metric="rmse",
        n_jobs=-1,
        **params,
    )


# ---------- Main ----------

def main():
    # 1) Load features
    print(f"Loading feature file: {FEATURE_FILE}")
    df = pd.read_csv(FEATURE_FILE)

    # Sort by time to be safe
    if "system_time" in df.columns:
        df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
        df = df.sort_values("system_time").reset_index(drop=True)

    print(f"Loaded features: {df.shape[0]} rows, {df.shape[1]} columns")

    # 2) Recreate y_rv_5s_log / rv_5s_past split as in cv_rv_5sec_xgb.py
    if "y_rv_5s_log" not in df.columns:
        raise ValueError("Column 'y_rv_5s_log' not found in features file.")
    if "rv_5s_past" not in df.columns:
        raise ValueError("Column 'rv_5s_past' not found in features file.")

    before = len(df)
    df = df.dropna(subset=["y_rv_5s_log", "rv_5s_past"]).reset_index(drop=True)
    after = len(df)
    print(f"Dropped {before - after} rows with NaN y_rv_5s_log / rv_5s_past; remaining: {after}")

    n = len(df)
    dev_end = int(n * 0.85)

    dev = df.iloc[:dev_end].copy()
    test = df.iloc[dev_end:].copy()

    print("\nGlobal split (for calibration):")
    print(f"  DEV:  {len(dev)} rows")
    print(f"  TEST: {len(test)} rows")

    # Targets
    y_dev = dev["y_rv_5s_log"].values
    y_test = test["y_rv_5s_log"].values

    # Baseline log(rv_5s_past)
    log_base_dev = np.log(dev["rv_5s_past"].values + EPS)
    log_base_test = np.log(test["rv_5s_past"].values + EPS)

    # 3) Load CV best config (params + alpha)
    print(f"\nLoading CV config from {CV_CONFIG_FILE}")
    with open(CV_CONFIG_FILE, "r") as f:
        cfg = json.load(f)

    # Be robust to two possible key patterns
    if "best_params" in cfg:
        best_params = cfg["best_params"]
        best_alpha = cfg.get("best_alpha", 0.35)
    else:
        best_params = cfg["params"]
        best_alpha = cfg.get("alpha", 0.35)

    print("Best params:", best_params)
    print(f"Best alpha (blend weight for baseline): {best_alpha:.2f}")

    # 4) Define features (same exclusions as CV 5s script)
    exclude_cols = {
        "system_time",
        "y_rv_1s", "y_rv_1s_log",
        "y_rv_5s", "y_rv_5s_log",
        "y_rv_10s", "y_rv_10s_log",
        "y_rv_1s_future", "y_rv_1s_future_log",
    }

    feature_cols = [c for c in df.columns if c not in exclude_cols]
    print(f"\nUsing {len(feature_cols)} features for 5s RV calibration.")

    X_dev = dev[feature_cols].values
    X_test = test[feature_cols].values

    # 5) Train XGB on DEV with best params
    print("\nTraining final 5s XGB on DEV with best params...")
    model = build_xgb(best_params)
    model.fit(
        X_dev,
        y_dev,
        eval_set=[(X_dev, y_dev)],
        verbose=False,
    )

    # 6) Compute raw blended log-RV on DEV and TEST
    log_xgb_dev = model.predict(X_dev)
    log_xgb_test = model.predict(X_test)

    log_raw_dev = best_alpha * log_base_dev + (1.0 - best_alpha) * log_xgb_dev
    log_raw_test = best_alpha * log_base_test + (1.0 - best_alpha) * log_xgb_test

    print("\n=== RAW BLEND PERFORMANCE (before calibration) ===")
    eval_log_and_rv(y_dev, log_raw_dev, name="DEV (raw blend)")
    eval_log_and_rv(y_test, log_raw_test, name="TEST (raw blend)")

    # 7) Fit IsotonicRegression calibrator on DEV
    print("\nFitting IsotonicRegression calibrator on DEV...")
    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(log_raw_dev, y_dev)

    # Apply calibrator
    log_cal_dev = iso.predict(log_raw_dev)
    log_cal_test = iso.predict(log_raw_test)

    print("\n=== CALIBRATED BLEND PERFORMANCE (after calibration) ===")
    eval_log_and_rv(y_dev, log_cal_dev, name="DEV (calibrated blend)")
    eval_log_and_rv(y_test, log_cal_test, name="TEST (calibrated blend)")

    # 8) Save calibrator (joblib so backtest can use joblib.load)
    joblib.dump(iso, "calibrator_rv_5s_isotonic.pkl")
    print("\nSaved calibrator to calibrator_rv_5s_isotonic.pkl")

    # 9) Diagnostic plot: mapping raw -> calibrated vs true (on DEV)
    print("Saving calibrator_rv_5s_isotonic.png ...")
    plt.figure(figsize=(6, 6))
    # Scatter: raw vs true (downsampled for plotting)
    plt.scatter(
        log_raw_dev[::100],
        y_dev[::100],
        s=5,
        alpha=0.3,
        label="DEV (raw vs true)",
    )
    # Isotonic mapping line
    order = np.argsort(log_raw_dev)
    plt.plot(
        log_raw_dev[order],
        iso.predict(log_raw_dev[order]),
        linewidth=2,
        label="Isotonic mapping (raw -> calibrated)",
    )
    plt.xlabel("Raw blended log-RV (log_raw)")
    plt.ylabel("Target log-RV (y_rv_5s_log)")
    plt.title("Isotonic calibrator for 5s log-RV")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("calibrator_rv_5s_isotonic.png")
    plt.close()
    print("Done. Diagnostic plot saved as calibrator_rv_5s_isotonic.png")

    # 10) Brief summary (TEST, RV-space)
    print("\n=== SUMMARY (TEST, RV-space) ===")
    y_rv_test = np.exp(y_test) - EPS
    y_rv_raw = np.exp(log_raw_test) - EPS
    y_rv_cal = np.exp(log_cal_test) - EPS

    rmse_raw = np.sqrt(mean_squared_error(y_rv_test, y_rv_raw))
    rmse_cal = np.sqrt(mean_squared_error(y_rv_test, y_rv_cal))
    corr_raw = np.corrcoef(y_rv_test, y_rv_raw)[0, 1]
    corr_cal = np.corrcoef(y_rv_test, y_rv_cal)[0, 1]

    print(f"Raw blend:    Corr_P={corr_raw:.4f}, RMSE_rv={rmse_raw:.3e}")
    print(f"Calibrated:   Corr_P={corr_cal:.4f}, RMSE_rv={rmse_cal:.3e}")


if __name__ == "__main__":
    main()


Writing calibrate_rv_5s.py


In [15]:
!python calibrate_rv_5s.py

Loading feature file: BTC_1sec_features.csv
Loaded features: 1030718 rows, 246 columns
Dropped 4 rows with NaN y_rv_5s_log / rv_5s_past; remaining: 1030714

Global split (for calibration):
  DEV:  876106 rows
  TEST: 154608 rows

Loading CV config from best_rv_5s_cv_config.json
Best params: {'n_estimators': 380, 'learning_rate': 0.021697713783363635, 'max_depth': 7, 'subsample': 0.7935133228268233, 'colsample_bytree': 0.8560204063533432, 'min_child_weight': 5.0}
Best alpha (blend weight for baseline): 0.35

Using 239 features for 5s RV calibration.

Training final 5s XGB on DEV with best params...
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:42:15] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device

In [16]:
%%writefile backtest_rv_5sec_varswap.py
#!/usr/bin/env python3
"""
backtest_rv_5sec_varswap.py

Toy "variance-swap style" backtest for 5-second realized variance (RV),
using the calibrated 5s RV model you already built.

Setup (matches cv_rv_5sec_xgb.py + calibrate_rv_5s.py):

- Input features:
    BTC_1sec_features.csv

- Target:
    y_rv_5s_log  (forward 5s RV in log-space, from feature_builder_1sec.py)

- Baseline:
    rv_5s_past   (realized RV over previous 5s window)
    log_baseline = log(rv_5s_past + eps)

- Model:
    XGB with best hyperparameters from best_rv_5s_cv_config.json
    Blend in log-space:
        log_blend = alpha * log_baseline + (1-alpha) * log_pred_xgb
    Then global isotonic calibrator (fitted on DEV):
        log_calibrated = iso(log_blend)

- Data split:
    DEV  = first 85% of rows (used for CV + final fit + calibrator fit)
    TEST = last 15% of rows (pure holdout)

Trading toys:

1) Simple sign varswap:
       pos_t = sign(forecast - baseline)

2) Magnitude-aware varswap with position sizing:
       edge_rel = (forecast - baseline) / (baseline + eps)
       if |edge_rel| < edge_min: pos = 0
       else: pos = clip(edge_rel / s_edge, -L_MAX, L_MAX)
       then regime scaling via rv_regime and gamma_map.

We then:

- Evaluate forecasts (baseline, XGB, blend, calibrated).
- Run simple sign varswap on DEV/TEST.
- Run magnitude-aware varswap once with default params.
- Run DEV-only grid search over (s_edge, edge_min, L_MAX) for the
  calibrated blend, pick the best Sharpe_5s, and apply that config to TEST.
"""

import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

FEATURE_FILE = "BTC_1sec_features.csv"
CV_CONFIG_FILE = "best_rv_5s_cv_config.json"
CALIB_FILE = "calibrator_rv_5s_isotonic.pkl"
EPS = 1e-18


# ---------- Helpers ----------

def spearman_corr(a, b):
    a_rank = pd.Series(a).rank()
    b_rank = pd.Series(b).rank()
    return a_rank.corr(b_rank)


def eval_log_and_rv(y_log_true, y_log_pred, name="set"):
    """Evaluate metrics in log-space and RV-space."""
    mse_log = mean_squared_error(y_log_true, y_log_pred)
    mae_log = mean_absolute_error(y_log_true, y_log_pred)
    corr_log = np.corrcoef(y_log_true, y_log_pred)[0, 1]

    # Back to RV space
    y_rv_true = np.exp(y_log_true) - EPS
    y_rv_pred = np.exp(y_log_pred) - EPS

    mse_rv = mean_squared_error(y_rv_true, y_rv_pred)
    mae_rv = mean_absolute_error(y_rv_true, y_rv_pred)
    corr_rv_p = np.corrcoef(y_rv_true, y_rv_pred)[0, 1]
    corr_rv_s = spearman_corr(y_rv_true, y_rv_pred)

    print(f"\n=== Forecast performance on {name} ===")
    print("Log-space:")
    print(f"  RMSE_log: {np.sqrt(mse_log):.6f}")
    print(f"  MAE_log:  {mae_log:.6f}")
    print(f"  Corr_log (Pearson): {corr_log:.4f}")
    print("RV-space:")
    print(f"  RMSE_rv:   {np.sqrt(mse_rv):.6e}")
    print(f"  MAE_rv:    {mae_rv:.6e}")
    print(f"  Corr_rv_P: {corr_rv_p:.4f}")
    print(f"  Corr_rv_S: {corr_rv_s:.4f}")

    return {
        "RMSE_log": float(np.sqrt(mse_log)),
        "MAE_log": float(mae_log),
        "RMSE_rv": float(np.sqrt(mse_rv)),
        "MAE_rv": float(mae_rv),
        "Corr_log": float(corr_log),
        "Corr_P": float(corr_rv_p),
        "Corr_S": float(corr_rv_s),
    }


def build_xgb(params):
    """Create an XGBRegressor from a params dict (GPU-enabled)."""
    return XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        device="cuda",
        eval_metric="rmse",
        n_jobs=-1,
        **params,
    )


def pnl_stats(pnl, name="strategy"):
    """Compute simple per-5s PnL stats."""
    pnl = np.asarray(pnl)
    mean = float(pnl.mean())
    std = float(pnl.std(ddof=0))
    sharpe = float(mean / std) if std > 0 else np.nan  # per-5s Sharpe
    hit_ratio = float((pnl > 0).mean())
    total = float(pnl.sum())
    print(f"\n=== PnL stats for {name} ===")
    print(f"  mean_pnl:    {mean:.6e}")
    print(f"  std_pnl:     {std:.6e}")
    print(f"  sharpe_5s:   {sharpe:.4f}  (per 5-second step)")
    print(f"  hit_ratio:   {hit_ratio:.4f}")
    print(f"  total_pnl:   {total:.6e}")
    return {
        "mean": mean,
        "std": std,
        "sharpe_5s": sharpe,
        "hit_ratio": hit_ratio,
        "total": total,
    }


def varswap_pnl(sigma2_real, sigma2_base, sigma2_forecast):
    """
    Simple variance-swap-style PnL per step for a given forecast.

        signal_t = forecast - baseline
        pos_t    = sign(signal_t)
        pnl_t    = pos_t * (realized - baseline)

    Inputs should all be in RV space (not log).
    """
    sigma2_real = np.asarray(sigma2_real)
    sigma2_base = np.asarray(sigma2_base)
    sigma2_forecast = np.asarray(sigma2_forecast)

    signal = sigma2_forecast - sigma2_base
    pos = np.sign(signal)
    pnl = pos * (sigma2_real - sigma2_base)
    return pnl


def varswap_pnl_with_pos(sigma2_real, sigma2_base, pos):
    """
    Variance-swap-style PnL given an externally provided position series:

        pnl_t = pos_t * (sigma2_real_t - sigma2_base_t)
    """
    sigma2_real = np.asarray(sigma2_real)
    sigma2_base = np.asarray(sigma2_base)
    pos = np.asarray(pos)
    pnl = pos * (sigma2_real - sigma2_base)
    return pnl


def build_magaware_positions(
    rv_forecast,
    rv_baseline,
    rv_regime=None,
    s_edge=0.20,
    edge_min=0.05,
    L_max=1.5,
    gamma_map=None,
):
    """
    Magnitude-aware, regime-aware position sizing.

    Inputs:
        rv_forecast : array-like, model forecast (RV space).
        rv_baseline : array-like, baseline "strike" (RV space).
        rv_regime   : array-like or None, integer volatility regime labels.
        s_edge      : scale where edge_rel/s_edge = 1 means full size.
        edge_min    : minimum |edge_rel| to take any position.
        L_max       : cap on absolute position size.
        gamma_map   : dict mapping regime -> multiplier (e.g. {0:1.0,1:0.7,2:0.4}).

    Returns:
        pos : np.ndarray of positions.
    """
    rv_forecast = np.asarray(rv_forecast)
    rv_baseline = np.asarray(rv_baseline)

    # Relative edge
    edge_rel = (rv_forecast - rv_baseline) / (rv_baseline + EPS)

    pos = np.zeros_like(edge_rel)

    # Only trade when |edge| >= edge_min
    mask = np.abs(edge_rel) >= edge_min
    pos[mask] = edge_rel[mask] / s_edge
    pos = np.clip(pos, -L_max, L_max)

    # Regime scaling
    if rv_regime is not None and gamma_map is not None:
        rv_regime = np.asarray(rv_regime)
        gammas = np.ones_like(pos, dtype=float)
        for reg, g in gamma_map.items():
            gammas[rv_regime == reg] = g
        pos = pos * gammas

    return pos


# ---------- Main ----------

def main():
    print(f"Loading feature file: {FEATURE_FILE}")
    df = pd.read_csv(FEATURE_FILE)

    # Sort by time to be safe
    if "system_time" in df.columns:
        df["system_time"] = pd.to_datetime(df["system_time"], utc=True)
        df = df.sort_values("system_time").reset_index(drop=True)

    print(f"Loaded features: {df.shape[0]} rows, {df.shape[1]} columns")

    # --- Target and baseline for 5s RV ---
    if "y_rv_5s_log" not in df.columns:
        raise ValueError("Column 'y_rv_5s_log' not found in features file.")
    if "rv_5s_past" not in df.columns:
        raise ValueError("Column 'rv_5s_past' not found (needed for baseline).")

    before = len(df)
    df = df.dropna(subset=["y_rv_5s_log", "rv_5s_past"]).reset_index(drop=True)
    after = len(df)
    print(f"Dropped {before - after} rows with NaN y_rv_5s_log / rv_5s_past; remaining: {after}")

    n = len(df)

    # Global DEV / TEST split (must match cv script)
    test_start = int(n * 0.85)
    dev = df.iloc[:test_start].copy()
    test = df.iloc[test_start:].copy()

    print("\nGlobal split:")
    print(f"  DEV:  {len(dev)} rows")
    print(f"  TEST: {len(test)} rows")

    # Baseline feature: log(rv_5s_past)
    dev_log_rv_past = np.log(dev["rv_5s_past"] + EPS)
    test_log_rv_past = np.log(test["rv_5s_past"] + EPS)

    # Targets (log-RV) and true RV in 5s space
    y_dev = dev["y_rv_5s_log"].values
    y_test = test["y_rv_5s_log"].values
    rv_dev_true = np.exp(y_dev) - EPS
    rv_test_true = np.exp(y_test) - EPS

    # --------- Load best CV config + calibrator ---------
    print(f"\nLoading CV config from {CV_CONFIG_FILE}")
    with open(CV_CONFIG_FILE, "r") as f:
        cv_conf = json.load(f)

    best_params = cv_conf["best_params"]
    best_alpha = float(cv_conf["best_alpha"])
    print("Best params:", best_params)
    print("Best alpha (blend weight for baseline):", best_alpha)

    print(f"\nLoading global isotonic calibrator from {CALIB_FILE}")
    iso = joblib.load(CALIB_FILE)


    # --------- Define feature columns (match cv script) ---------
    exclude_cols = {
        "system_time",
        "y_rv_1s", "y_rv_1s_log",
        "y_rv_5s", "y_rv_5s_log",
        "y_rv_10s", "y_rv_10s_log",
        "y_rv_1s_future", "y_rv_1s_future_log",
    }
    feature_cols = [c for c in dev.columns if c not in exclude_cols]
    print(f"\nUsing {len(feature_cols)} features for 5s RV.")

    X_dev = dev[feature_cols].values
    X_test = test[feature_cols].values

    # Regime column (if available)
    dev_regime = dev["rv_regime"].values if "rv_regime" in dev.columns else None
    test_regime = test["rv_regime"].values if "rv_regime" in test.columns else None

    # --------- Train final XGB on DEV ---------
    print("\nTraining final 5s XGB on DEV with best params (GPU if available)...")
    model = build_xgb(best_params)
    model.fit(
        X_dev,
        y_dev,
        eval_set=[(X_dev, y_dev)],
        verbose=False,
    )

    # --------- FORECASTS: DEV ---------
    log_base_dev = dev_log_rv_past.values
    log_xgb_dev = model.predict(X_dev)
    log_blend_dev = best_alpha * log_base_dev + (1.0 - best_alpha) * log_xgb_dev
    log_cal_dev = iso.predict(log_blend_dev)

    # Forecast evaluation on DEV
    _ = eval_log_and_rv(y_dev, log_base_dev, name="DEV (baseline forecast)")
    _ = eval_log_and_rv(y_dev, log_xgb_dev, name="DEV (XGB forecast)")
    _ = eval_log_and_rv(y_dev, log_blend_dev, name=f"DEV (blend, alpha={best_alpha:.2f})")
    _ = eval_log_and_rv(y_dev, log_cal_dev, name="DEV (blend + global isotonic)")

    # convert to RV space
    rv_dev_base = np.exp(log_base_dev) - EPS
    rv_dev_xgb = np.exp(log_xgb_dev) - EPS
    rv_dev_blend = np.exp(log_blend_dev) - EPS
    rv_dev_cal = np.exp(log_cal_dev) - EPS

    # --------- FORECASTS: TEST ---------
    log_base_test = test_log_rv_past.values
    log_xgb_test = model.predict(X_test)
    log_blend_test = best_alpha * log_base_test + (1.0 - best_alpha) * log_xgb_test
    log_cal_test = iso.predict(log_blend_test)

    _ = eval_log_and_rv(y_test, log_base_test, name="TEST (baseline forecast)")
    _ = eval_log_and_rv(y_test, log_xgb_test, name="TEST (XGB forecast)")
    _ = eval_log_and_rv(y_test, log_blend_test, name=f"TEST (blend, alpha={best_alpha:.2f})")
    _ = eval_log_and_rv(y_test, log_cal_test, name="TEST (blend + global isotonic)")

    rv_test_base = np.exp(log_base_test) - EPS
    rv_test_xgb = np.exp(log_xgb_test) - EPS
    rv_test_blend = np.exp(log_blend_test) - EPS
    rv_test_cal = np.exp(log_cal_test) - EPS

    # --------- VARIANCE-SWAP-STYLE BACKTEST (SIMPLE SIGN) ---------
    print("\n========== VARIANCE-SWAP STYLE BACKTEST (SIGN STRATEGY) ==========")
    print("PnL definition:")
    print("  pos_t = sign( sigma2_forecast_t - sigma2_baseline_t )")
    print("  PnL_t = pos_t * ( sigma2_realized_t - sigma2_baseline_t )")
    print("where baseline = rv_5s_past, forecast in {XGB, blend, calibrated}.\n")

    # DEV PnL (simple sign)
    pnl_dev_xgb = varswap_pnl(rv_dev_true, rv_dev_base, rv_dev_xgb)
    pnl_dev_blend = varswap_pnl(rv_dev_true, rv_dev_base, rv_dev_blend)
    pnl_dev_cal = varswap_pnl(rv_dev_true, rv_dev_base, rv_dev_cal)

    stats_dev_xgb = pnl_stats(pnl_dev_xgb, name="DEV (XGB varswap)")
    stats_dev_blend = pnl_stats(pnl_dev_blend, name="DEV (blend varswap)")
    stats_dev_cal = pnl_stats(pnl_dev_cal, name="DEV (calibrated blend varswap)")

    # TEST PnL (simple sign)
    pnl_test_xgb = varswap_pnl(rv_test_true, rv_test_base, rv_test_xgb)
    pnl_test_blend = varswap_pnl(rv_test_true, rv_test_base, rv_test_blend)
    pnl_test_cal = varswap_pnl(rv_test_true, rv_test_base, rv_test_cal)

    stats_test_xgb = pnl_stats(pnl_test_xgb, name="TEST (XGB varswap)")
    stats_test_blend = pnl_stats(pnl_test_blend, name="TEST (blend varswap)")
    stats_test_cal = pnl_stats(pnl_test_cal, name="TEST (calibrated blend varswap)")

    print("\nSaving cumulative PnL plot for TEST (simple sign) as varswap_pnl_5s_test.png ...")
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pnl_test_xgb), label="XGB varswap (sign)")
    plt.plot(np.cumsum(pnl_test_blend), label=f"Blend varswap (sign, alpha={best_alpha:.2f})")
    plt.plot(np.cumsum(pnl_test_cal), label="Calibrated blend varswap (sign)")
    plt.axhline(0.0, linestyle="--", linewidth=1)
    plt.title("Variance-swap style cumulative PnL (5s RV, TEST, sign strategy)")
    plt.xlabel("5-second steps (TEST)")
    plt.ylabel("Cumulative PnL (RV units)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("varswap_pnl_5s_test.png")
    plt.close()
    print("Done simple PnL plot.")

    # --------- MAGNITUDE-AWARE VARIANCE-SWAP STRATEGY (SINGLE RUN) ---------
    print("\n========== MAGNITUDE-AWARE VARIANCE-SWAP STRATEGY ==========")

    # Default gamma_map for regimes 0,1,2
    default_gamma_map = {0: 1.0, 1: 0.7, 2: 0.4}

    if dev_regime is not None:
        print("  [DEV] Using rv_regime for regime-aware scaling.")
    else:
        print("  [DEV] No rv_regime column; using gamma = 1.0 everywhere.")
        default_gamma_map = None

    if test_regime is not None:
        print("  [TEST] Using rv_regime for regime-aware scaling.")
    else:
        print("  [TEST] No rv_regime column; using gamma = 1.0 everywhere.")

    # Default parameters (the ones we previously used)
    s_edge_default = 0.20
    edge_min_default = 0.05
    L_max_default = 1.5

    pos_dev_mag = build_magaware_positions(
        rv_forecast=rv_dev_cal,
        rv_baseline=rv_dev_base,
        rv_regime=dev_regime,
        s_edge=s_edge_default,
        edge_min=edge_min_default,
        L_max=L_max_default,
        gamma_map=default_gamma_map,
    )
    pnl_dev_mag = varswap_pnl_with_pos(rv_dev_true, rv_dev_base, pos_dev_mag)
    stats_dev_mag = pnl_stats(pnl_dev_mag, name="DEV (calibrated blend magnitude-aware varswap)")

    pos_test_mag = build_magaware_positions(
        rv_forecast=rv_test_cal,
        rv_baseline=rv_test_base,
        rv_regime=test_regime,
        s_edge=s_edge_default,
        edge_min=edge_min_default,
        L_max=L_max_default,
        gamma_map=default_gamma_map,
    )
    pnl_test_mag = varswap_pnl_with_pos(rv_test_true, rv_test_base, pos_test_mag)
    stats_test_mag = pnl_stats(pnl_test_mag, name="TEST (calibrated blend magnitude-aware varswap)")

    print("\nSaving cumulative PnL plot for TEST (magnitude-aware) as varswap_pnl_5s_test_magaware.png ...")
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pnl_test_mag), label="Calibrated blend (mag-aware)")
    plt.axhline(0.0, linestyle="--", linewidth=1)
    plt.title("Magnitude-aware variance-swap cumulative PnL (5s RV, TEST)")
    plt.xlabel("5-second steps (TEST)")
    plt.ylabel("Cumulative PnL (RV units)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("varswap_pnl_5s_test_magaware.png")
    plt.close()
    print("Done magnitude-aware PnL plot.")

    # --------- GRID SEARCH ON DEV FOR MAGNITUDE-AWARE STRATEGY ---------
    print("\n========== GRID SEARCH (DEV ONLY) FOR MAGNITUDE-AWARE STRATEGY ==========")
    print("Searching over s_edge, edge_min, L_MAX; keeping gamma_map fixed (if available).")

    # Parameter grids
    s_edge_grid = [0.10, 0.15, 0.20, 0.30]
    edge_min_grid = [0.02, 0.05, 0.08]
    L_max_grid = [1.0, 1.5, 2.0]

    best_cfg = None
    best_sharpe = -np.inf

    for s_edge in s_edge_grid:
        for edge_min in edge_min_grid:
            for L_max in L_max_grid:
                pos_dev_grid = build_magaware_positions(
                    rv_forecast=rv_dev_cal,
                    rv_baseline=rv_dev_base,
                    rv_regime=dev_regime,
                    s_edge=s_edge,
                    edge_min=edge_min,
                    L_max=L_max,
                    gamma_map=default_gamma_map,
                )
                pnl_dev_grid = varswap_pnl_with_pos(rv_dev_true, rv_dev_base, pos_dev_grid)
                mean = pnl_dev_grid.mean()
                std = pnl_dev_grid.std(ddof=0)
                sharpe = float(mean / std) if std > 0 else -np.inf

                print(f"  [DEV] s_edge={s_edge:.2f}, edge_min={edge_min:.2f}, L_max={L_max:.2f} "
                      f"-> Sharpe_5s={sharpe:.4f}")

                if sharpe > best_sharpe:
                    best_sharpe = sharpe
                    best_cfg = {
                        "s_edge": s_edge,
                        "edge_min": edge_min,
                        "L_max": L_max,
                    }

    print("\nBest DEV config (magnitude-aware, fixed gamma_map):")
    print(best_cfg)
    print(f"Best DEV Sharpe_5s: {best_sharpe:.4f}")

    # Apply best config to DEV and TEST for final reporting
    s_edge_best = best_cfg["s_edge"]
    edge_min_best = best_cfg["edge_min"]
    L_max_best = best_cfg["L_max"]

    pos_dev_best = build_magaware_positions(
        rv_forecast=rv_dev_cal,
        rv_baseline=rv_dev_base,
        rv_regime=dev_regime,
        s_edge=s_edge_best,
        edge_min=edge_min_best,
        L_max=L_max_best,
        gamma_map=default_gamma_map,
    )
    pnl_dev_best = varswap_pnl_with_pos(rv_dev_true, rv_dev_base, pos_dev_best)
    stats_dev_best = pnl_stats(pnl_dev_best, name="DEV (mag-aware, grid-search best)")

    pos_test_best = build_magaware_positions(
        rv_forecast=rv_test_cal,
        rv_baseline=rv_test_base,
        rv_regime=test_regime,
        s_edge=s_edge_best,
        edge_min=edge_min_best,
        L_max=L_max_best,
        gamma_map=default_gamma_map,
    )
    pnl_test_best = varswap_pnl_with_pos(rv_test_true, rv_test_base, pos_test_best)
    stats_test_best = pnl_stats(pnl_test_best, name="TEST (mag-aware, grid-search best)")

    print("\nSaving cumulative PnL plot for TEST (mag-aware, grid-search best) "
          "as varswap_pnl_5s_test_magaware_best.png ...")
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pnl_test_best), label="Mag-aware (grid-search best)")
    plt.axhline(0.0, linestyle="--", linewidth=1)
    plt.title("Magnitude-aware varswap cumulative PnL (5s RV, TEST, grid-search best)")
    plt.xlabel("5-second steps (TEST)")
    plt.ylabel("Cumulative PnL (RV units)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("varswap_pnl_5s_test_magaware_best.png")
    plt.close()
    print("Done mag-aware best PnL plot.")

    # --------- Tiny JSON summary (including new best strategy) ---------
    summary = {
        "dev": {
            "xgb_sign": stats_dev_xgb,
            "blend_sign": stats_dev_blend,
            "calibrated_sign": stats_dev_cal,
            "calibrated_magaware_default": stats_dev_mag,
            "calibrated_magaware_best": stats_dev_best,
        },
        "test": {
            "xgb_sign": stats_test_xgb,
            "blend_sign": stats_test_blend,
            "calibrated_sign": stats_test_cal,
            "calibrated_magaware_default": stats_test_mag,
            "calibrated_magaware_best": stats_test_best,
        },
        "best_magaware_params": {
            "s_edge": s_edge_best,
            "edge_min": edge_min_best,
            "L_max": L_max_best,
            "dev_sharpe_5s": best_sharpe,
        },
    }
    with open("varswap_pnl_5s_summary.json", "w") as f:
        json.dump(summary, f, indent=2)
    print("Saved varswap_pnl_5s_summary.json")


if __name__ == "__main__":
    main()


Overwriting backtest_rv_5sec_varswap.py


In [17]:
!python backtest_rv_5sec_varswap.py

Loading feature file: BTC_1sec_features.csv
Loaded features: 1030718 rows, 246 columns
Dropped 4 rows with NaN y_rv_5s_log / rv_5s_past; remaining: 1030714

Global split:
  DEV:  876106 rows
  TEST: 154608 rows

Loading CV config from best_rv_5s_cv_config.json
Best params: {'n_estimators': 380, 'learning_rate': 0.021697713783363635, 'max_depth': 7, 'subsample': 0.7935133228268233, 'colsample_bytree': 0.8560204063533432, 'min_child_weight': 5.0}
Best alpha (blend weight for baseline): 0.35000000000000003

Loading global isotonic calibrator from calibrator_rv_5s_isotonic.pkl

Using 239 features for 5s RV.

Training final 5s XGB on DEV with best params (GPU if available)...
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:44:33] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is o